import requests
from bs4 import BeautifulSoup
import pandas as pd
import plotly.graph_objects as go
import re

# ---------------------------------------------------------
# HIGHLIGHT COUNTRIES MANUALLY
# ---------------------------------------------------------
USER_COUNTRY = "Philippines"  # red
USER_COLOR = "red"

HIGHLIGHT_COUNTRY = "Japan"  # orange now
HIGHLIGHT_COLOR = "orange"

# ---------------------------------------------------------
# FETCH NATIONAL DEBT DATA
# ---------------------------------------------------------
url = "https://worldpopulationreview.com/country-rankings/countries-by-national-debt"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

table = soup.find("table")
rows = table.find_all("tr")[1:]

DEBT_MULT = {"T": 1e12, "B": 1e9, "M": 1e6}

records = []

for row in rows:
    cols = row.find_all("td")
    if len(cols) < 5:
        continue

    country = cols[1].text.strip()
    if country.upper() == "TOTAL":
        continue

    debt_text = cols[2].text.strip()
    gdp_pct_text = cols[3].text.strip()
    per_capita_text = cols[4].text.strip()

    # Debt USD
    m = re.match(r"\$(\d+(\.\d+)?)([TBM])", debt_text)
    if not m:
        continue
    debt_val = float(m.group(1))
    debt_unit = m.group(3)
    debt_usd = int(debt_val * DEBT_MULT[debt_unit])

    # Debt % GDP
    debt_pct_gdp = float(gdp_pct_text.replace("%", "")) if "%" in gdp_pct_text else None

    # Debt per Capita
    per_capita = None
    pc_text = per_capita_text.replace("$", "").replace(",", "").strip()
    pc_match = re.match(r"(\d+(\.\d+)?)\s*(Mn|Bn)?", pc_text)
    if pc_match:
        pc_val = float(pc_match.group(1))
        pc_unit = pc_match.group(3)
        if pc_unit == "Mn":
            per_capita = int(pc_val * 1_000_000)
        elif pc_unit == "Bn":
            per_capita = int(pc_val * 1_000_000_000)
        else:
            per_capita = int(pc_val)

    records.append({
        "Country": country,
        "Debt Label": debt_text,
        "Debt USD": debt_usd,
        "Debt % GDP": debt_pct_gdp,
        "Debt Per Capita": per_capita
    })

df = pd.DataFrame(records)

# ---------------------------------------------------------
# HELPER TO CREATE RANK LABELS
# ---------------------------------------------------------
def create_rank_labels(df, value_col, value_format="raw"):
    df_sorted = df.sort_values(value_col, ascending=False).reset_index(drop=True)
    df_sorted["Rank"] = df_sorted.index + 1

    def format_value(row):
        if value_format == "debt":
            return f"{row['Debt Label']}"
        elif value_format == "pct":
            return f"{row[value_col]}%"
        elif value_format == "percap":
            val = row[value_col]
            if val >= 1e9:
                return f"${val/1e9:.2f} B"
            elif val >= 1e6:
                return f"${val/1e6:.2f} M"
            else:
                return f"${val}"
        else:
            return str(row[value_col])

    df_sorted["Plot Label"] = df_sorted.apply(
        lambda x: f"rank {x['Rank']}, {format_value(x)}", axis=1
    )
    return df_sorted.sort_values(value_col)

# ---------------------------------------------------------
# HELPERS TO HIGHLIGHT COUNTRIES
# ---------------------------------------------------------
def build_bar_colors(df, country_col="Country", user_country=None, user_color=None,
                     extra_country=None, extra_color=None):
    colors = []
    for c in df[country_col]:
        if user_country and c == user_country:
            colors.append(user_color)
        elif extra_country and c == extra_country:
            colors.append(extra_color)
        else:
            colors.append("#1f77b4")
    return colors

def build_text_colors(df, country_col="Country", user_country=None, user_color=None,
                     extra_country=None, extra_color=None):
    colors = []
    for c in df[country_col]:
        if user_country and c == user_country:
            colors.append(user_color)
        elif extra_country and c == extra_country:
            colors.append(extra_color)
        else:
            colors.append("black")
    return colors

# ---------------------------------------------------------
# NATIONAL DEBT CHART
# ---------------------------------------------------------
df1_plot = create_rank_labels(df, "Debt USD", "debt")
fig1 = go.Figure(go.Bar(
    x=df1_plot["Debt USD"],
    y=df1_plot["Country"],
    orientation="h",
    text=df1_plot["Plot Label"],
    textposition="outside",
    marker=dict(
        color=build_bar_colors(df1_plot, user_country=USER_COUNTRY, user_color=USER_COLOR,
                               extra_country=HIGHLIGHT_COUNTRY, extra_color=HIGHLIGHT_COLOR)
    ),
    textfont=dict(
        color=build_text_colors(df1_plot, user_country=USER_COUNTRY, user_color=USER_COLOR,
                                extra_country=HIGHLIGHT_COUNTRY, extra_color=HIGHLIGHT_COLOR)
    )
))
fig1.update_layout(
    title="National Debt by Country",
    xaxis_title="National Debt (USD)",
    height=max(600, len(df1_plot)*28),
    margin=dict(l=220, r=320),
    xaxis=dict(range=[0, df1_plot["Debt USD"].max()*1.3]),
    template="plotly_white"
)
fig1.show()

# ---------------------------------------------------------
# DEBT AS % OF GDP CHART
# ---------------------------------------------------------
df2 = df.dropna(subset=["Debt % GDP"])
df2_plot = create_rank_labels(df2, "Debt % GDP", "pct")
fig2 = go.Figure(go.Bar(
    x=df2_plot["Debt % GDP"],
    y=df2_plot["Country"],
    orientation="h",
    text=df2_plot["Plot Label"],
    textposition="outside",
    marker=dict(
        color=build_bar_colors(df2_plot, user_country=USER_COUNTRY, user_color=USER_COLOR,
                               extra_country=HIGHLIGHT_COUNTRY, extra_color=HIGHLIGHT_COLOR)
    ),
    textfont=dict(
        color=build_text_colors(df2_plot, user_country=USER_COUNTRY, user_color=USER_COLOR,
                                extra_country=HIGHLIGHT_COUNTRY, extra_color=HIGHLIGHT_COLOR)
    )
))
fig2.update_layout(
    title="Debt as Percentage of GDP",
    xaxis_title="Debt (% of GDP)",
    height=max(600, len(df2_plot)*28),
    margin=dict(l=220, r=200),
    xaxis=dict(range=[0, df2_plot["Debt % GDP"].max()*1.2]),
    template="plotly_white"
)
fig2.show()

# ---------------------------------------------------------
# DEBT PER CAPITA CHART
# ---------------------------------------------------------
df3 = df.dropna(subset=["Debt Per Capita"])
df3_plot = create_rank_labels(df3, "Debt Per Capita", "percap")
fig3 = go.Figure(go.Bar(
    x=df3_plot["Debt Per Capita"],
    y=df3_plot["Country"],
    orientation="h",
    text=df3_plot["Plot Label"],
    textposition="outside",
    marker=dict(
        color=build_bar_colors(df3_plot, user_country=USER_COUNTRY, user_color=USER_COLOR,
                               extra_country=HIGHLIGHT_COUNTRY, extra_color=HIGHLIGHT_COLOR)
    ),
    textfont=dict(
        color=build_text_colors(df3_plot, user_country=USER_COUNTRY, user_color=USER_COLOR,
                                extra_country=HIGHLIGHT_COUNTRY, extra_color=HIGHLIGHT_COLOR)
    )
))
fig3.update_layout(
    title="Debt Per Capita by Country",
    xaxis_title="Debt Per Capita (USD)",
    height=max(600, len(df3_plot)*28),
    margin=dict(l=220, r=240),
    xaxis=dict(range=[0, df3_plot["Debt Per Capita"].max()*1.2]),
    template="plotly_white"
)
fig3.show()

# ---------------------------------------------------------
# SUMMARY DATAFRAME WITH PERCENTAGE CALCULATIONS
# ---------------------------------------------------------
main_country = USER_COUNTRY
compare_country = HIGHLIGHT_COUNTRY

main_row = df[df["Country"] == main_country]
compare_row = df[df["Country"] == compare_country]

def calculate_percentage_metrics(val1, val2):
    if val1 is None or val2 is None:
        return None, None, None
    larger = max(val1, val2)
    smaller = min(val1, val2)
    perc_increase = (larger / smaller - 1) * 100
    perc_decrease = (1 - smaller / larger) * 100
    perc_diff = abs(larger - smaller) / ((larger + smaller)/2) * 100
    return perc_increase, perc_decrease, perc_diff

summary_data = {
    "Metric": ["National Debt (USD)", "Debt % GDP", "Debt Per Capita"]
}

metrics_main = [
    int(main_row["Debt USD"].iloc[0]) if not main_row.empty else None,
    float(main_row["Debt % GDP"].iloc[0]) if not main_row.empty else None,
    int(main_row["Debt Per Capita"].iloc[0]) if not main_row.empty else None
]

metrics_compare = [
    int(compare_row["Debt USD"].iloc[0]) if not compare_row.empty else None,
    float(compare_row["Debt % GDP"].iloc[0]) if not compare_row.empty else None,
    int(compare_row["Debt Per Capita"].iloc[0]) if not compare_row.empty else None
]

diff_list = [abs(a - b) if a is not None and b is not None else None for a,b in zip(metrics_main, metrics_compare)]

# Calculate percentage metrics
perc_inc_list = []
perc_dec_list = []
perc_diff_list = []

for a, b in zip(metrics_main, metrics_compare):
    inc, dec, diff = calculate_percentage_metrics(a, b)
    perc_inc_list.append(inc)
    perc_dec_list.append(dec)
    perc_diff_list.append(diff)

# Format values
summary_data[main_country] = [
    f"${metrics_main[0]:,}" if metrics_main[0] is not None else None,
    f"{metrics_main[1]:.2f}%" if metrics_main[1] is not None else None,
    f"${metrics_main[2]:,}" if metrics_main[2] is not None else None
]

summary_data[compare_country] = [
    f"${metrics_compare[0]:,}" if metrics_compare[0] is not None else None,
    f"{metrics_compare[1]:.2f}%" if metrics_compare[1] is not None else None,
    f"${metrics_compare[2]:,}" if metrics_compare[2] is not None else None
]

summary_data["Difference"] = [
    f"${diff_list[0]:,}" if diff_list[0] is not None else None,
    f"{diff_list[1]:.2f}%" if diff_list[1] is not None else None,
    f"${diff_list[2]:,}" if diff_list[2] is not None else None
]

summary_data["Percentage Increase"] = [f"{x:.2f}%" if x is not None else None for x in perc_inc_list]
summary_data["Percentage Decrease"] = [f"{x:.2f}%" if x is not None else None for x in perc_dec_list]
summary_data["Percentage Difference"] = [f"{x:.2f}%" if x is not None else None for x in perc_diff_list]

# Create DataFrame
summary_df = pd.DataFrame(summary_data)

# Display full width in Jupyter Notebook
pd.set_option('display.expand_frame_repr', False)

print("\n--- Country Debt Comparison ---")
print(summary_df)
